In [1]:
import numpy as np
import scipy, h5py
import scipy.stats as stats
import math
import ROOT
import os,sys
import tables
import uproot, argparse
import scipy.io as scio
from scipy.optimize import minimize
from scipy import special

/home/orv/gentoo/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Welcome to JupyROOT 6.18/02


In [2]:
# physical constant
Light_yield = 4285*0.88 # light yield
Att_LS = 18 # attenuation length of LS
Att_Wtr = 300 # attenuation length of water
tau_r = 1.6 # fast time constant
TTS = 5.5/2.355
QE = 0.20
PMT_radius = 0.254
c = 2.99792e8
n = 1.48
shell = 0.65 # Acrylic

In [12]:
def Likelihood(fit, *args):
    Energy,\
    x,\
    y,\
    z,\
    t,\
    tau_d\
    = fit
    PMT_pos, pe_array, time_array, fired_PMT= args
    distance, Omega = SolidAngle(x,y,z)
    lmbd = Att(x,y,z)
    # expect photons
    expect = Energy*\
        Light_yield*\
        np.exp(-distance*lmbd/Att_LS - distance*(1-lmbd)/Att_Wtr)*\
        Omega*\
        QE
    # log Poisson # p_pe = - np.log(stats.poisson.pmf(PE, expect))
    log_p_pe = - expect + pe_array*np.log(expect) 
    # this part is nonsense {- np.log(special.factorial(pe_array))}
    Likelihood_pe = - np.nansum(log_p_pe)
    # log Time profile pdf
    # log_p_time = TimeProfile(time_array, distance[fired_PMT], tau_d, t)
    # Likelihood_time = - np.nansum(log_p_time)
    # total likelihood
    Likelihood_total = Likelihood_pe
    #Likelihood_total = Likelihood_pe + Likelihood_time
    return Likelihood_total

In [4]:
def SolidAngle(x, y, z):
    distance = np.sqrt(np.sum((PMT_pos - np.array((x,y,z)))**2, axis=1))
    radius_O1 = PMT_radius # PMT bottom surface
    PMT_vector = - PMT_pos/np.transpose(np.tile(np.sqrt(np.sum(PMT_pos**2,1)),[3,1]))
    O1 = np.tile(np.array([x,y,z]),[len(PMT_pos[:,0]),1])
    O2 = PMT_pos
    flight_vector = O2 - O1
    d2 = np.sqrt(np.sum(flight_vector**2,1))
    theta1 = np.sum(PMT_vector*flight_vector,1)/np.sqrt(np.sum(PMT_vector**2,1)*np.sum(flight_vector**2,1))
    Omega = (1-d2/np.sqrt(d2**2+radius_O1*np.abs(theta1)))/2
    
    return distance, Omega

In [5]:
def Att(x, y, z):
    '''
    this function returns ratio in different material 
    lmbd is in the LS and 1-lmbda is the water
    '''
    # LS distance
    d1 = np.tile(np.array([x,y,z]),[len(PMT_pos[:,1]),1])
    d2 = PMT_pos
    d3 = d2 - d1
    # cons beyond shell 
    lmbd = (-2*np.sum(d3*d1,1) \
        + np.sqrt(4*np.sum(d3*d1,1)**2 \
        - 4*np.sum(d3**2,1)*(-np.abs((np.sum(d1**2,1)-shell**2))))) \
        /(2*np.sum(d3**2,1))
    lmbd[lmbd>=1] = 1
    return lmbd

In [6]:
def TimeProfile(time_array, distance, tau_d, t):
    time_correct = time_array - distance/(c/n)*1e9 - t
    time_correct[time_correct<=-8] = -8
    p_time = TimeUncertainty(time_correct, tau_d)
    return p_time

In [7]:
def TimeUncertainty(tc, tau_d):
    a1 = np.exp(((TTS**2 - tc*tau_d)**2-tc**2*tau_d**2)/(2*TTS**2*tau_d**2))
    a2 = np.exp(((TTS**2*(tau_d+tau_r) - tc*tau_d*tau_r)**2 - tc**2*tau_d**2*tau_r**2)/(2*TTS**2*tau_d**2*tau_r**2))
    a3 = np.exp(((TTS**2 - tc*tau_d)**2 - tc**2*tau_d**2)/(2*TTS**2*tau_d**2))*special.erf((tc*tau_d-TTS**2)/(np.sqrt(2)*tau_d*TTS))
    a4 = np.exp(((TTS**2*(tau_d+tau_r) - tc*tau_d*tau_r)**2 - tc**2*tau_d**2*tau_r**2)/(2*TTS**2*tau_d**2*tau_r**2))*special.erf((tc*tau_d*tau_r-TTS**2*(tau_d+tau_r))/(np.sqrt(2)*tau_d*tau_r*TTS))
    p_time  = np.log(tau_d + tau_r) - 2*np.log(tau_d) + np.log(a1-a2+a3-a4)
    
    return p_time

In [8]:
def con(args):
    E_min,\
    E_max,\
    tau_min,\
    tau_max,\
    t0_min,\
    t0_max\
    = args
    cons = ({'type': 'ineq', 'fun': lambda x: (x[0] - E_min)*(E_max - x[0])},\
    {'type': 'ineq', 'fun': lambda x: shell**2 - (x[1]**2 + x[2]**2 + x[3]**2)},\
    {'type': 'ineq', 'fun': lambda x: (x[5] - tau_min)*(tau_max-x[5])},\
    {'type': 'ineq', 'fun': lambda x: (x[4] - t0_min)*(t0_max-x[4])})
    return cons

In [9]:
def recon_drc(time_array, fired_PMT, recon_vertex):
    time_corr = time_array - np.sum(PMT_pos[fired_PMT,1:4]-np.tile(recon_vertex[0,1:4],[len(fired_PMT),1]))/(3*10**8)
    index = np.argsort(time_corr)
    fired_PMT_sorted = fired_PMT[index]
    fired_PMT_sorted = fired_PMT_sorted[0:int(np.floor(len(fired_PMT_sorted)/10))]
    drc = np.sum(PMT_pos[fired_PMT_sorted,1:4],0)/len(fired_PMT_sorted)
    return drc

In [10]:
def ReadPMT():
    f = open(r"./PMT1t.txt")
    line = f.readline()
    data_list = [] 
    while line:
        num = list(map(float,line.split()))
        data_list.append(num)
        line = f.readline()
    f.close()
    PMT_pos = np.array(data_list)
    return PMT_pos

In [13]:
def recon(fid, fout, *args):
    PMT_pos, event_count,shut = args
    # global event_count,shell,PE,time_array,PMT_pos, fired_PMT
    '''
    reconstruction

    fid: root reference file
    fout: output file
    '''
    # Create the output file and the group
    rootfile = ROOT.TFile(fid)
    print(fid) # filename
    class ReconData(tables.IsDescription):
        EventID = tables.Int64Col(pos=0)    # EventNo
        x1 = tables.Float16Col(pos=1)        # x position
        y1 = tables.Float16Col(pos=2)        # y position
        z1 = tables.Float16Col(pos=3)        # z position
        x2 = tables.Float16Col(pos=4)        # x position
        y2 = tables.Float16Col(pos=5)        # y position
        z2 = tables.Float16Col(pos=6)        # z position
        x3 = tables.Float16Col(pos=7)        # x position
        y3 = tables.Float16Col(pos=8)        # y position
        z3 = tables.Float16Col(pos=9)        # z position
        x4 = tables.Float16Col(pos=10)        # x position
        y4 = tables.Float16Col(pos=11)        # y position
        z4 = tables.Float16Col(pos=12)        # z position
    # Create the output file and the group
    h5file = tables.open_file(fout, mode="w", title="OneTonDetector",
                            filters = tables.Filters(complevel=9))
    group = "/"
    # Create tables
    ReconTable = h5file.create_table(group, "Recon", ReconData, "Recon")
    recondata = ReconTable.row
    # Loop for event
    f = uproot.open(fid)
    a = f['SimpleAnalysis']
    for tot, chl, PEl, Pkl, nPl in zip(a.array("TotalPE"),  # total pe in an event
                    a.array("ChannelInfo.ChannelId"),       # PMT fired seq
                    a.array('ChannelInfo.PE'),              # Hit info number on PMT
                    a.array('ChannelInfo.PeakLoc'),         # Time info on PMT
                    a.array('ChannelInfo.nPeaks')):         # 
        pe_array = np.zeros(np.size(PMT_pos[:,1])) # Photons on each PMT (PMT size * 1 vector)
        fired_PMT = np.zeros(0)     # Hit PMT (PMT Seq can be repeated)
        time_array = np.zeros(0, dtype=int)    # Time info (Hit number)
        for ch, pe, pk, npk in zip(chl, PEl, Pkl, nPl):
            pe_array[ch] = pe
            time_array = np.hstack((time_array, pk))
            fired_PMT = np.hstack((fired_PMT, ch*np.ones(np.size(pk))))
        fired_PMT = fired_PMT.astype(int)
        # initial result
        result_vertex = np.empty((0,6)) # reconstructed vertex
        # initial value x[0] = [1,6]
        x0 = np.zeros((1,4))
        x0[0][0] = pe_array.sum()/300
        x0[0][1] = np.sum(pe_array*PMT_pos[:,0])/np.sum(pe_array)
        x0[0][2] = np.sum(pe_array*PMT_pos[:,1])/np.sum(pe_array)
        x0[0][3] = np.sum(pe_array*PMT_pos[:,2])/np.sum(pe_array)
        # cut 1 PMT
        x1 = np.zeros((1,6))
        pe_array_tmp = pe_array
        pe_array_tmp[shut] = 0
        x1[0][0] = pe_array.sum()/300
        x1[0][1] = np.sum(pe_array*PMT_pos[:,0])/np.sum(pe_array)
        x1[0][2] = np.sum(pe_array*PMT_pos[:,1])/np.sum(pe_array)
        x1[0][3] = np.sum(pe_array*PMT_pos[:,2])/np.sum(pe_array)
        # Constraints
        E_min = 0.01
        E_max = 100
        tau_min = 0.01
        tau_max = 100
        t0_min = -300
        t0_max = 300
        con_args = E_min, E_max, tau_min, tau_max, t0_min, t0_max
        cons = con(con_args)
        # reconstruction
        result1 = minimize(Likelihood, x0, method='SLSQP', constraints=cons, \
        args = (PMT_pos, pe_array, time_array, fired_PMT))
        result2 = minimize(Likelihood, x0, method='SLSQP', constraints=cons, \
        args = (PMT_pos, pe_array_tmp, time_array, fired_PMT))
        # result
        print(event_count, result.x, result.success)
        event_count = event_count + 1
        recondata['EventID'] = event_count
        recondata['x1'] = result.x1[1]
        recondata['y1'] = result.x1[2]
        recondata['z1'] = result.x1[3]
        recondata['x2'] = result.x2[1]
        recondata['y2'] = result.x2[2]
        recondata['z2'] = result.x2[3]
        recondata['x3'] = x0[0][1]
        recondata['y3'] = x0[0][2]
        recondata['z3'] = x0[0][3]
        recondata['x4'] = x1[0][1]
        recondata['y4'] = x1[0][2]
        recondata['z4'] = x1[0][3]
        recondata.append()
    # Flush into the output file
    ReconTable.flush()
    h5file.close()

In [15]:
if len(sys.argv)!=4:
    print("Wront arguments!")
    print("Usage: python Recon.py MCFileName[.root] outputFileName[.h5] shut")
    sys.exit(1)
# Read PMT position
PMT_pos = ReadPMT()
event_count = 0

ROOT.PyConfig.IgnoreCommandLineOptions = True
# Reconstruction
fid = sys.argv[1] # input file .root
fout = sys.argv[2] # output file .h5
shut = sys.argv[3]
args = PMT_pos, event_count, shut
recon(fid, fout, *args)

Wront arguments!
Usage: python Recon.py MCFileName[.root] outputFileName[.h5] shut


SystemExit: 1

/home/orv/gentoo/usr/lib64/python3.6/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
